In [ ]:
import torch
import torchvision
import skimage.io as io
import clip
from PIL import Image
import pickle
import os
from tqdm import tqdm
import argparse
import pandas as pd
import numpy as np


device = torch.device('cuda:0')
clip_model_type = "ViT-B_32"
out_path = "./data/Video_CLIP_ViT-B_32.pkl"
frames_no = 10 
csv_path = "./Charades/Charades_v1_train.csv"
video_path = "./video" 
model, preprocess = clip.load(clip_model_type, device=device, jit=False)

# load captions and video names
df = pd.read_csv(csv_path)

#extract frames
def extract_frames(video_file, frames_no):
    video, audio, info = torchvision.io.read_video(video_file)

    num_frames = info["video_fps"] * info["video_duration"]
    frame_rate = info["video_fps"]
    interval = int(frame_rate * 60 / frames_no)
    frames = []
    to_pil = torchvision.transforms.ToPILImage()
    for i in range(0, num_frames, interval):
      frames.append(to_pil(video[i]))

    return frames

def encode_frames(frames):

    tensors = []

    for i in range(1,len(frames)):
        frames[i] = preprocess(frames[i]).unsqueeze(0).to(device)











#load videos


for i in tqdm(range(len(data))):
    d = data[i]
    img_id = d["image_id"]
    filename = f"./data/coco/train2014/COCO_train2014_{int(img_id):012d}.jpg"
    if not os.path.isfile(filename):
        filename = f"./data/coco/val2014/COCO_val2014_{int(img_id):012d}.jpg"
    image = io.imread(filename)
    image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
    with torch.no_grad():
        prefix = clip_model.encode_image(image).cpu()
    d["clip_embedding"] = i
    all_embeddings.append(prefix)
    all_captions.append(d)
    if (i + 1) % 10000 == 0:
        with open(out_path, 'wb') as f:
            pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

with open(out_path, 'wb') as f:
    pickle.dump({"clip_embedding": torch.cat(all_embeddings, dim=0), "captions": all_captions}, f)

print('Done')
print("%0d embeddings saved " % len(all_embeddings))
return 0




In [ ]:
# Import the libraries
import torch
import torchvision
import clip

# Define the parameters
x = 10 # Number of frames per minute to extract
video_path = "videos/" # Path to the folder containing the videos
output_path = "output/" # Path to the folder where the output tensors will be saved

# Load the CLIP model and preprocess
model, preprocess = clip.load("ViT-B/32", device="cuda")

# Define a function to extract frames from a video
def extract_frames(video_file, x):
  # Load the video using torchvision
  video, audio, info = torchvision.io.read_video(video_file)

  # Get the number of frames and the frame rate
  num_frames = info["video_fps"] * info["video_duration"]
  frame_rate = info["video_fps"]

  # Calculate the interval between frames to extract
  interval = int(frame_rate * 60 / x)

  # Initialize an empty list to store the extracted frames
  frames = []

  # Loop through the video and append every interval-th frame to the list
  for i in range(0, num_frames, interval):
    frames.append(video[i])

  # Return the list of frames
  return frames

"""# Define a function to encode frames using CLIP
def encode_frames(frames):
  # Initialize an empty list to store the encoded tensors
  tensors = []

  # Loop through the frames and preprocess them
  for frame in frames:
    # Resize and center crop the frame to 224x224 pixels
    frame = torchvision.transforms.Resize(224)(frame)
    frame = torchvision.transforms.CenterCrop(224)(frame)

    # Convert the frame to a tensor and normalize it
    frame = torchvision.transforms.ToTensor()(frame)
    frame = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))(frame)

    # Append the frame tensor to the list
    tensors.append(frame)

  # Stack the tensors into a batch
  batch = torch.stack(tensors)

  # Encode the batch using CLIP and get the image features
  with torch.no_grad():
    image_features = model.encode_image(batch)

  # Return the image features tensor
  return image_features"""

def encode_frames(frames):
  # Initialize an empty list to store the tensors
  tensors = []

  # Loop through the frames and preprocess them using CLIP
  for frame in frames:
    # Convert the frame to a tensor
    frame = torchvision.transforms.ToTensor()(frame)

    # Preprocess the frame using CLIP
    frame = preprocess(frame)

    # Append the frame tensor to the list
    tensors.append(frame)

  # Stack the tensors into a batch
  batch = torch.stack(tensors)

  # Encode the batch using CLIP and get the image features
  with torch.no_grad():
    image_features = model.encode_image(batch)

  # Return the image features tensor
  return image_features

# Define a function to concatenate tensors along a dimension
def concatenate_tensors(tensors, dim):
  # Stack the tensors along the specified dimension
  concatenated_tensor = torch.stack(tensors, dim=dim)

  # Return the concatenated tensor
  return concatenated_tensor

# Loop through the videos in the folder
for video_file in os.listdir(video_path):
  # Extract x frames per minute from the video
  frames = extract_frames(video_path + video_file, x)

  # Encode the frames using CLIP
  image_features = encode_frames(frames)

  # Concatenate the image features along dimension 1 to make a 2D tensor
  output_tensor = concatenate_tensors(image_features, dim=1)

  # Save the output tensor to a file with the same name as the video file
  torch.save(output_tensor, output_path + video_file + ".pt")